## i) Import packages/libraries

In [2]:
import os
import glob
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
master_dr = os.path.split(os.getcwd())[0]
# change the directory to where the modules are saved
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

In [3]:
roi_shapefile= os.path.join(master_dr, r"Data/1Boundary/Shapefile/wheat.shp")
gdf= gpd.read_file(roi_shapefile)
name_column = 'FID'

In [4]:
data_df = gdf[[name_column, 'geometry']]
data_df[name_column] = data_df[name_column].astype(str)
data_df.loc[len(data_df), name_column] = 'all_area'
seasonal_df = data_df.copy()

In [5]:
seasonal_dr = os.path.join(master_dr, r"Data/tif/seasonal")
seasonal_tifs = glob.glob(seasonal_dr+'/*.tif')
monthly_dr = os.path.join(master_dr, r"Data/tif/monthly")
# data_names = [os.path.basename(path).split('.')[0].split('_')[1].upper() for path in seasonal_tifs]

## ii) Import the input data

In [6]:
# get seasonal data
AETI_seasonal    = os.path.join(seasonal_dr, f"seasonal_AETI.tif")

## Calculate uniformity of water consumption
* Equity is defined as the coefficients of variation (CV) of seasonal ETa in the area of interest.
* It measures the evenness of the water supply in an irrigation scheme. 
* Note: CV of 0 to 10% is good, 10 to 25% is fair and CV > 25% is poor uniformity (Bastiaanssen et al., 1996) 
<br/>

In [7]:
#Uniformity of water Consumption 

# calculate for the whole area
src = rasterio.open(AETI_seasonal)
array = rasterio.mask.mask(src, gdf.geometry, crop=True, filled=True, nodata= np.nan)[0][0]
AETIm   = np.nanmean(array)
AETIsd  = np.nanstd(array)
CV_AETI = (AETIsd / AETIm) * 100
if CV_AETI < 10:
    U = 'Good'
if (CV_AETI >= 10) and (CV_AETI < 25):
    U = 'Fair'
else: 
    U = 'Poor'
seasonal_df.loc[seasonal_df[name_column]=='all_area', 'UNIFORMITY'] = CV_AETI
seasonal_df.loc[seasonal_df[name_column]=='all_area', 'UNIFORMITY CLASS'] = U

# for every feature
for i,row in gdf.iterrows():
    geometry = [row['geometry']]
    array = rasterio.mask.mask(src, geometry, crop=True, filled=True, nodata= np.nan)[0][0]
    AETIm   = np.nanmean(array)
    AETIsd  = np.nanstd(array)
    CV_AETI = (AETIsd / AETIm) * 100
    if CV_AETI < 10:
        U = 'Good'
    if (CV_AETI >= 10) and (CV_AETI < 25):
        U = 'Fair'
    else: 
        U = 'Poor'
    seasonal_df.loc[seasonal_df[name_column]==str(row[name_column]), 'UNIFORMITY'] = CV_AETI
    seasonal_df.loc[seasonal_df[name_column]==str(row[name_column]), 'UNIFORMITY CLASS'] = U


## Step 3e - Calculate Relative water Deficit (RWD)
$RWD= 1-\frac{ET_a}{ET_x}$
<br/>${ET_x} = $ Can be ETp or 99 percentile of the actual evapotranspiration

In [8]:
# collecting Geoinfo such as projection, the x and y axis
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(AETI_seasonal)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster



# calculate for the whole area
src = rasterio.open(AETI_seasonal)
AETI = rasterio.mask.mask(src, gdf.geometry, crop=True, filled=True, nodata= np.nan)[0][0]
AETI1_1D  = np.reshape(AETI,  AETI.shape[0]*AETI.shape[1])
ETx       = np.nanpercentile(AETI1_1D, 99)
AETI_mean = np.nanmean(AETI)
RWD = 1-(AETI_mean/ETx)
seasonal_df.loc[seasonal_df[name_column]=='all_area', 'RWD'] = round(RWD, 2)

# for every feature
for i,row in gdf.iterrows():
    geometry = [row['geometry']]
    AETI = rasterio.mask.mask(src, geometry, crop=True, filled=True, nodata= np.nan)[0][0]
    AETI1_1D  = np.reshape(AETI,  AETI.shape[0]*AETI.shape[1])
    ETx       = np.nanpercentile(AETI1_1D, 99)
    AETI_mean = np.nanmean(AETI)
    RWD = 1-(AETI_mean/ETx)
    seasonal_df.loc[seasonal_df[name_column]==str(row[name_column]), 'RWD'] = round(RWD, 2)

## Calculate tifs for every feature (area)

In [9]:
# seasonal
for tif in seasonal_tifs:
  data_name = os.path.basename(tif).split('.')[0].split('_')[1].upper()
  # calculate for the whole area
  src = rasterio.open(tif)
  array = rasterio.mask.mask(src, gdf.geometry, crop=True, filled=True, nodata= np.nan)[0][0]
  mean = np.nanmedian(array)
  seasonal_df.loc[seasonal_df[name_column]=='all_area', data_name] = mean
  # for every feature
  for i,row in gdf.iterrows():
    geometry = [row['geometry']]
    array = rasterio.mask.mask(src, geometry, crop=True, filled=True, nodata= np.nan)[0][0]
    mean = np.nanmedian(array)
    seasonal_df.loc[seasonal_df[name_column]==str(row[name_column]), data_name] = mean

In [10]:
# monthly
data_names = sorted(os.listdir(monthly_dr))
months = ["_".join(i.split(".")[0].split("_")[-2:]) for i in sorted(os.listdir(f"{monthly_dr}/AETI"))]
months_dfs = []
for month in months:
  month_tifs = [f"{monthly_dr}/{n}/monthly_{n}_{month}.tif" for n in data_names]
  month_df = data_df.copy()
  for tif in month_tifs:
    data_name = os.path.basename(tif).split('.')[0].split('_')[1].upper()
    # calculate for the whole area
    src = rasterio.open(tif)
    array = rasterio.mask.mask(src, gdf.geometry, crop=True, filled=True, nodata= np.nan)[0][0]
    mean = np.nanmedian(array)
    month_df.loc[month_df[name_column]=='all_area', data_name] = mean
    # for every feature
    for i,row in gdf.iterrows():
      geometry = [row['geometry']]
      array = rasterio.mask.mask(src, geometry, crop=True, filled=True, nodata= np.nan)[0][0]
      mean = np.nanmedian(array)
      month_df.loc[month_df[name_column]==str(row[name_column]), data_name] = mean
  months_dfs.append(month_df)

In [11]:
# save data as sheet
data_sheets_dr = os.path.join(master_dr, r"Data/datasheets")
os.makedirs(data_sheets_dr, exist_ok= True)

# seasonal
data_df_dr = os.path.join(data_sheets_dr, "seasonal.csv")
df_to_save = seasonal_df.copy().drop('geometry',axis=1)
df_to_save.to_csv(data_df_dr, index=False)
# monthly
for df, month in zip(months_dfs, months):
  data_df_dr = os.path.join(data_sheets_dr, f"monthly_{month}.csv")
  df_to_save = df.copy().drop('geometry',axis=1)
  df_to_save.to_csv(data_df_dr, index=False)

In [12]:
# save data as vectors
vectors_dr = os.path.join(master_dr, r"Data/vectors")
os.makedirs(vectors_dr, exist_ok= True)

# seasonal
df_to_save = seasonal_df.copy()
df_to_save = df_to_save.drop(df_to_save[df_to_save[name_column] == 'all_area'].index, axis=0)
df_to_save = gpd.GeoDataFrame(df_to_save, geometry=df_to_save["geometry"])
data_df_dr = os.path.join(vectors_dr, "seasonal.geojson")
df_to_save.to_file(data_df_dr, driver="GeoJSON")
# monthly
for df, month in zip(months_dfs, months):
  df_to_save = df.copy()
  df_to_save = df_to_save.drop(df_to_save[df_to_save[name_column] == 'all_area'].index, axis=0)
  df_to_save = gpd.GeoDataFrame(df_to_save, geometry=df_to_save["geometry"])
  data_df_dr = os.path.join(vectors_dr, f"monthly_{month}.geojson")
  df_to_save.to_file(data_df_dr, driver="GeoJSON")